### Models expect a batch of inputs

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = torch.tensor(ids)
# This line will fail.
model(input_ids)

c:\Users\FPTSHOP\miniconda3\envs\aio2024-exercise\Lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Users\FPTSHOP\miniconda3\envs\aio2024-exercise\Lib\site-packages\huggingface_hub\file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\FPTSHOP\miniconda3\envs\aio2024-exercise\Lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Users\FPTSHOP\miniconda3\envs\aio2024-exercise\Lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils.

IndexError: too many indices for tensor of dimension 1

In [2]:
print(ids)

[1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012]


In [ ]:
tokenized_inputs = tokenizer(sequence, return_tensors="pt")
print(tokenized_inputs["input_ids"])

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)

input_ids = torch.tensor([ids])
print("Input IDs:", input_ids)

output = model(input_ids)
print("Logits:", output.logits)

c:\Users\FPTSHOP\miniconda3\envs\aio2024-exercise\Lib\site-packages\huggingface_hub\file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Input IDs: tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012]])
Logits: tensor([[-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)


### Padding the inputs
Đưa các sequence không có chung độ dài về 1 độ dài

In [1]:
batched_ids = [
    [200, 200, 200],
    [200, 200]
]

padding_id = 100

batched_ids = [
    [200, 200, 200],
    [200, 200, padding_id],
]

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence1_ids = [[200, 200, 200]]
sequence2_ids = [[200, 200]]
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

print(model(torch.tensor(sequence1_ids)).logits)
print(model(torch.tensor(sequence2_ids)).logits)
print(model(torch.tensor(batched_ids)).logits)

### Attention masks
Ẩn đi ngẫu nhiên một số token

In [3]:
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

attention_mask = [
    [1, 1, 1],
    [1, 1, 0],
]

outputs = model(torch.tensor(batched_ids), attention_mask=torch.tensor(attention_mask))
print(outputs.logits)

tensor([[ 1.5694, -1.3895],
        [ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)


### Example

In [ ]:
sequences = ["I've been waiting for a HuggingFace course my whole life.", 
            "I hate this so much!"]
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

c:\Users\FPTSHOP\miniconda3\envs\aio2024-exercise\Lib\site-packages\huggingface_hub\file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [27]:
tokens = tokenizer(sequences, padding=True, truncation=True, return_tensors='pt')
# attention_mask = torch.where(tokens['input_ids'] > 0, 1, 0)
model(tokens['input_ids'])

SequenceClassifierOutput(loss=None, logits=tensor([[-1.5607,  1.6123],
        [ 3.4603, -2.9194]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [38]:
def create_attn_mask(input_ids, p=0.2):
    attn_mask = torch.rand((input_ids.shape))
    return torch.where(attn_mask <= p, 0, 1)

attention_mask = create_attn_mask(tokens['input_ids'])
model(tokens['input_ids'], attention_mask)

SequenceClassifierOutput(loss=None, logits=tensor([[-2.1629,  2.2103],
        [ 3.6176, -3.0246]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)